In [1]:
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from IPython.display import Image
from IPython.core.display import HTML 
import time
from ipywidgets import FloatProgress
from IPython.display import display
from bs4 import BeautifulSoup
import requests
import re
from os import path
from PIL import Image
import pandas as pd
import warnings
warnings.filterwarnings("ignore")
from models import *
from etl import *
import plotly.plotly as py
import plotly.graph_objs as go

Database session created


In [2]:
def add_scraping_source(df, identifier_dict):
    scraped_source = []
    for index, row in df.iterrows():
        identifier = row.medium + row.formality
        scraped_from = identifier_dict[identifier]
        scraped_source.append(scraped_from)
    df['scraped_from'] = scraped_source
    return df
def fix_NYT(df):
    df.loc[(df.provider=='The New York Times') & (df.medium == 'text'), 'scraped_from'] = 'NYT API'
    return df

In [3]:
identifier_dict = {'audioFormal':'Itunes Audiobook','textFormal':'Itunes Ebook','videoFormal':'Itunes Movie',
                  'textInformal': 'Twitter', 'videoInformal': 'Youtube',
                   'audioIntermediate': 'Itunes Podcast', 'textIntermediate': 'NewsApi', 'videoIntermediate': 'Youtube'}

In [4]:
pd.set_option('display.max_colwidth', -1)

In [5]:
df = pd.DataFrame([{'id':content.id,'content_url':content.content_url, 'image_url':content.image_url,'title':content.title
                             , 'published': content.published, 'length': content.length, 'medium':content.medium.name,
                             'provider': content.provider.name, 'api_id':content.provider.api_id,
                    'formality':content.provider.formality.type, 'difficulty':content.difficulty.type
                             } for content in session.query(Content).all()])
x = [[category.name for category in content.categories] for content in session.query(Content).all()]
df['categories'] = x
df = add_scraping_source(df, identifier_dict)
df = fix_NYT(df)

df.to_csv('all_content.csv')

In [6]:
df

,api_id,content_url,difficulty,formality,id,image_url,length,medium,provider,published,title,categories,scraped_from
0,Twitter,https://twitter.com/ATPScience1/status/1075922927575556097,Hard,Informal,1,https://twitter.com/ATPScience1/status/1075922927575556097,1,text,ATPScience1,2018-12-21,1075922927575556097,"[general, null]",Twitter
1,Twitter,https://twitter.com/ATPScience1/status/1075922863402700802,Medium,Informal,2,https://twitter.com/ATPScience1/status/1075922863402700802,1,text,ATPScience1,2018-12-21,1075922863402700802,"[general, null]",Twitter
2,Twitter,https://twitter.com/ATPScience1/status/1075921037483741184,Easy,Informal,3,https://twitter.com/ATPScience1/status/1075921037483741184,1,text,ATPScience1,2018-12-21,1075921037483741184,"[general, null]",Twitter
3,Twitter,https://twitter.com/ATPScience1/status/1075920493595611137,Easy,Informal,4,https://twitter.com/ATPScience1/status/1075920493595611137,1,text,ATPScience1,2018-12-21,1075920493595611137,"[null, aminos, vegan]",Twitter
4,Twitter,https://twitter.com/ATPScience1/status/1075917766530486274,Hard,Informal,5,https://twitter.com/ATPScience1/status/1075917766530486274,1,text,ATPScience1,2018-12-21,1075917766530486274,"[null, aminos, vegan]",Twitter
5,Twitter,https://twitter.com/ATPScience1/status/1075908604635549700,Medium,Informal,6,https://twitter.com/ATPScience1/status/1075908604635549700,1,text,ATPScience1,2018-12-21,1075908604635549700,"[general, null]",Twitter
6,Twitter,https://twitter.com/ATPScience1/status/1075878801987399681,Medium,Informal,7,https://twitter.com/ATPScience1/status/1075878801987399681,1,text,ATPScience1,2018-12-20,1075878801987399681,"[general, null]",Twitter
7,Twitter,https://twitter.com/ATPScience1/status/1075878740289298432,Easy,Informal,8,https://twitter.com/ATPScience1/status/1075878740289298432,1,text,ATPScience1,2018-12-20,1075878740289298432,"[general, null]",Twitter
8,Twitter,https://twitter.com/ATPScience1/status/1073412341075726336,Easy,Informal,9,https://twitter.com/ATPScience1/status/1073412341075726336,1,text,ATPScience1,2018-12-14,1073412341075726336,"[general, null]",Twitter
9,Twitter,https://twitter.com/ATPScience1/status/1073400673461506048,Hard,Informal,10,https://twitter.com/ATPScience1/status/1073400673461506048,1,text,ATPScience1,2018-12-14,1073400673461506048,"[general, null]",Twitter


In [7]:
x = [[category.name for category in content.categories] for content in session.query(Content).all()]
y = [l for l in x if len(l) > 2]

In [8]:
y

[['null', 'aminos', 'vegan'],
 ['null', 'aminos', 'vegan'],
 ['null', 'gut', 'microbiome'],
 ['null', 'gut', 'microbiome'],
 ['null', 'vegan', 'meal prep'],
 ['null', 'vegan', 'meal prep'],
 ['null', 'vegan', 'beans'],
 ['null', 'vegan', 'cooking'],
 ['null', 'plant', 'meat'],
 ['null', 'vegan', 'vegan diet'],
 ['null', 'carbs', 'fiber'],
 ['null', 'ketogenic diet', 'ketogenic'],
 ['null', 'ketogenic diet', 'ketogenic'],
 ['null', 'gut', 'probiotic'],
 ['null', 'gut', 'microbiome'],
 ['null', 'gut', 'sugar'],
 ['null', 'gut', 'microbiome'],
 ['null', 'gut', 'microbiome'],
 ['null', 'gut', 'gut health'],
 ['null', 'gut', 'probiotic'],
 ['null', 'gut', 'microbiome'],
 ['null', 'gut', 'gut health'],
 ['null', 'sugar', 'paleo'],
 ['null', 'paleo', 'sprouts'],
 ['null', 'protein', 'amino', 'animal protein', 'animal'],
 ['null', 'paleo', 'paleo diet'],
 ['null', 'paleo', 'paleo diet'],
 ['null', 'paleo', 'paleo diet'],
 ['null', 'keto', 'meat'],
 ['null', 'keto', 'meat'],
 ['null', 'keto', '

In [41]:
df.head(75)

,api_id,content_url,difficulty,formality,id,image_url,length,medium,provider,published,title,categories,scraped_from
0,575955428,https://twitter.com/ATPScience1/status/1075922927575556097,Easy,Informal,1,https://twitter.com/ATPScience1/status/1075922927575556097,1,text,ATPScience1,2018-12-21,1075922927575556097,"[general, null]",Twitter
1,575955428,https://twitter.com/ATPScience1/status/1075922863402700802,Easy,Informal,2,https://twitter.com/ATPScience1/status/1075922863402700802,1,text,ATPScience1,2018-12-21,1075922863402700802,"[general, null]",Twitter
2,575955428,https://twitter.com/ATPScience1/status/1075921037483741184,Easy,Informal,3,https://twitter.com/ATPScience1/status/1075921037483741184,1,text,ATPScience1,2018-12-21,1075921037483741184,"[general, null]",Twitter
3,575955428,https://twitter.com/ATPScience1/status/1075920493595611137,Easy,Informal,4,https://twitter.com/ATPScience1/status/1075920493595611137,1,text,ATPScience1,2018-12-21,1075920493595611137,"[general, null]",Twitter
4,575955428,https://twitter.com/ATPScience1/status/1075917766530486274,Easy,Informal,5,https://twitter.com/ATPScience1/status/1075917766530486274,1,text,ATPScience1,2018-12-21,1075917766530486274,"[general, null]",Twitter
5,575955428,https://twitter.com/ATPScience1/status/1075908604635549700,Easy,Informal,6,https://twitter.com/ATPScience1/status/1075908604635549700,1,text,ATPScience1,2018-12-21,1075908604635549700,"[general, null]",Twitter
6,575955428,https://twitter.com/ATPScience1/status/1075878801987399681,Easy,Informal,7,https://twitter.com/ATPScience1/status/1075878801987399681,1,text,ATPScience1,2018-12-20,1075878801987399681,"[general, null]",Twitter
7,575955428,https://twitter.com/ATPScience1/status/1075878740289298432,Easy,Informal,8,https://twitter.com/ATPScience1/status/1075878740289298432,1,text,ATPScience1,2018-12-20,1075878740289298432,"[general, null]",Twitter
8,575955428,https://twitter.com/ATPScience1/status/1073412341075726336,Easy,Informal,9,https://twitter.com/ATPScience1/status/1073412341075726336,1,text,ATPScience1,2018-12-14,1073412341075726336,"[general, null]",Twitter
9,575955428,https://twitter.com/ATPScience1/status/1073400673461506048,Easy,Informal,10,https://twitter.com/ATPScience1/status/1073400673461506048,1,text,ATPScience1,2018-12-14,1073400673461506048,"[general, null]",Twitter


# Total Content Per Medium & Formality

In [31]:
total = df[['content_url','formality','medium','scraped_from']].groupby(['formality','medium','scraped_from']).nunique()
total[['content_url']]

,,,content_url
formality,medium,scraped_from,
Informal,text,Twitter,1501


# Total Unique Categories and Providers per Medium & Formality

In [33]:
plot_stacked_OVERALL()

IndexError: list index out of range

# Ketogenic

In [34]:
Limit_widget1 = widgets.IntSlider(min=1, max=5, step=1, value=1)
Medium_widget1 = widgets.ToggleButtons(options = ['text', 'audio','video'], description ='Medium', button_style='')
topic_widget1 = widgets.Dropdown(options = nutrition_cats,value='keto', description ='Topic', disabled=False)
expertise_widget1 = widgets.ToggleButtons(options = ['Informal', 'Intermediate','Formal'], description ='Formality', button_style='')
max_length_widget1 = widgets.IntSlider(min=0, max=100, step=10, value=20)
difficulty_widget1 = widgets.ToggleButtons(options = ['Easy', 'Medium','Hard'], description ='Difficulty', button_style='')


interact(query_content,Limit = Limit_widget1, Medium = Medium_widget1, search_param=topic_widget1, Formality = expertise_widget1, Max_Length = max_length_widget1,Difficulty = difficulty_widget1);

interactive(children=(IntSlider(value=1, description='Limit', max=5, min=1), ToggleButtons(description='Medium…

In [11]:
alt_widget_1 = widgets.Dropdown(options = nutrition_cats,value='keto', description ='Topic', disabled=False)

interact(plot_stacked, param = alt_widget_1);

interactive(children=(Dropdown(description='Topic', options=('keto', 'mct', 'natural', 'corn-fed', 'cruciferou…

# Paleolithic

In [12]:
Limit_widget2 = widgets.IntSlider(min=1, max=5, step=1, value=1)
Medium_widget2 = widgets.ToggleButtons(options = ['text', 'audio','video'], description ='Medium', button_style='')
topic_widget2 = widgets.Dropdown(options = nutrition_cats,value='paleo', description ='Topic', disabled=False)
expertise_widget2 = widgets.ToggleButtons(options = ['Informal', 'Intermediate','Formal'], description ='Formality', button_style='')
max_length_widget2 = widgets.IntSlider(min=0, max=100, step=10, value=20)
difficulty_widget2 = widgets.ToggleButtons(options = ['Easy', 'Medium','Hard'], description ='Difficulty', button_style='')


interact(query_content,Limit = Limit_widget2, Medium = Medium_widget2, search_param=topic_widget2, Formality = expertise_widget2, Max_Length = max_length_widget2,Difficulty = difficulty_widget2);

interactive(children=(IntSlider(value=1, description='Limit', max=5, min=1), ToggleButtons(description='Medium…

In [13]:
alt_widget_2 = widgets.Dropdown(options = nutrition_cats,value='paleo', description ='Topic', disabled=False)

interact(plot_stacked, param = alt_widget_2);

interactive(children=(Dropdown(description='Topic', index=14, options=('keto', 'mct', 'natural', 'corn-fed', '…

# Vegan

In [13]:
Limit_widget3 = widgets.IntSlider(min=1, max=5, step=1, value=1)
Medium_widget3 = widgets.ToggleButtons(options = ['text', 'audio','video'], description ='Medium', button_style='')
topic_widget3 = widgets.Dropdown(options = nutrition_cats,value='vegan', description ='Topic', disabled=False)
expertise_widget3 = widgets.ToggleButtons(options = ['Informal', 'Intermediate','Formal'], description ='Formality', button_style='')
max_length_widget3 = widgets.IntSlider(min=0, max=100, step=10, value=20)
difficulty_widget3 = widgets.ToggleButtons(options = ['Easy', 'Medium','Hard'], description ='Difficulty', button_style='')


interact(query_content,Limit = Limit_widget3, Medium = Medium_widget3, search_param=topic_widget3, Formality = expertise_widget3, Max_Length = max_length_widget3, Difficulty = difficulty_widget3);

interactive(children=(IntSlider(value=1, description='Limit', max=5, min=1), ToggleButtons(description='Medium…

In [14]:
alt_widget_3 = widgets.Dropdown(options = nutrition_cats,value='vegan', description ='Topic', disabled=False)

interact(plot_stacked, param = alt_widget_3);

interactive(children=(Dropdown(description='Topic', index=13, options=('keto', 'mct', 'natural', 'corn-fed', '…

# Vegetarian

In [15]:
Limit_widget4 = widgets.IntSlider(min=1, max=5, step=1, value=1)
Medium_widget4 = widgets.ToggleButtons(options = ['text', 'audio','video'], description ='Medium', button_style='')
topic_widget4 = widgets.Dropdown(options = nutrition_cats,value='vegetarian', description ='Topic', disabled=False)
expertise_widget4 = widgets.ToggleButtons(options = ['Informal', 'Intermediate','Formal'], description ='Formality', button_style='')
max_length_widget4 = widgets.IntSlider(min=0, max=200, step=10, value=20)
difficulty_widget4 = widgets.ToggleButtons(options = ['Easy', 'Medium','Hard'], description ='Difficulty', button_style='')

interact(query_content,Limit = Limit_widget4, Medium = Medium_widget4, search_param=topic_widget4, Formality = expertise_widget4, Max_Length = max_length_widget4, Difficulty = difficulty_widget4);

interactive(children=(IntSlider(value=1, description='Limit', max=5, min=1), ToggleButtons(description='Medium…

In [16]:
alt_widget_4 = widgets.Dropdown(options = nutrition_cats,value='vegetarian', description ='Topic', disabled=False)

interact(plot_stacked, param = alt_widget_4);

interactive(children=(Dropdown(description='Topic', index=14, options=('keto', 'mct', 'natural', 'corn-fed', '…

# General

In [17]:
Limit_widget5 = widgets.IntSlider(min=1, max=5, step=1, value=1)
Medium_widget5 = widgets.ToggleButtons(options = ['text', 'audio','video'], description ='Medium', button_style='')
topic_widget5 = widgets.Dropdown(options = ['general'],value='general', description ='Topic', disabled=False)
expertise_widget5 = widgets.ToggleButtons(options = ['Informal', 'Intermediate','Formal'], description ='Formality', button_style='')
max_length_widget5 = widgets.IntSlider(min=0, max=300, step=10, value=20)
difficulty_widget5 = widgets.ToggleButtons(options = ['Easy', 'Medium','Hard'], description ='Difficulty', button_style='')

interact(query_content,Limit = Limit_widget5, Medium = Medium_widget5, search_param=topic_widget5, Formality = expertise_widget5, Max_Length = max_length_widget5,Difficulty - difficulty_widget5);

interactive(children=(IntSlider(value=1, description='Limit', max=5, min=1), ToggleButtons(description='Medium…

In [18]:
alt_widget_5 = widgets.Dropdown(options = ['general'],value='general', description ='Topic', disabled=False)

interact(plot_stacked, param = alt_widget_5);

interactive(children=(Dropdown(description='Topic', options=('general',), value='general'), Output()), _dom_cl…

# Search

In [19]:
# search_widget = widgets.Text(placeholder = 'Search News', description='Search:', disabled=False,continuous_update=False)
# Limit_widget5 = widgets.IntSlider(min=1, max=5, step=1, value=1)
# Medium_widget5 = widgets.ToggleButtons(options = ['text', 'audio','video'], description ='Medium', button_style='')




# interact(query_content,search_param=search_widget,Limit = Limit_widget5, Medium = Medium_widget5);